In [ ]:
from pathlib import Path
import duckdb, pandas as pd, numpy as np
import matplotlib.pyplot as plt

here = Path.cwd().resolve()
ROOT = None
for r in [here, *here.parents]:
    if (r / "data" / "processed").exists():
        ROOT = r
        break
if ROOT is None:
    raise FileNotFoundError(f"Couldn't find project root from {here}. "
                            "Expected a 'data/processed' folder.")

DATA = ROOT / "data" / "processed"
RPTS = ROOT / "reports"
RPTS.mkdir(exist_ok=True, parents=True)

con = duckdb.connect()
print("ROOT:", ROOT)
print("DATA:", DATA)

ROOT: C:\Users\maria\credit-risk
DATA: C:\Users\maria\credit-risk\data\processed


In [2]:
def q(sql: str, *args) -> pd.DataFrame:
    return con.execute(sql, args).df()

def savefig(name: str):
    p = RPTS / name
    plt.tight_layout()
    plt.savefig(p, dpi=140)
    plt.close()
    print("saved:", p)

In [3]:
vintage = q("""
SELECT vintage_q,
       COUNT(*) AS n,
       AVG(CASE WHEN default_within_24m THEN 1 ELSE 0 END)::DOUBLE AS bad_rate
FROM read_parquet($1)
GROUP BY 1 ORDER BY 1
""", str(DATA/"abt.parquet"))

display(vintage.head())
vintage.to_csv(RPTS/"eda_vintage.csv", index=False)

plt.figure()
plt.plot(vintage["vintage_q"], vintage["bad_rate"])
plt.xticks(rotation=45); plt.title("Bad rate by vintage")
plt.xlabel("vintage_q"); plt.ylabel("bad_rate")
savefig("eda_bad_rate_by_vintage.png")

,vintage_q,n,bad_rate
0,2017-01-01,86328,0.008549
1,2017-04-01,247238,0.009323
2,2017-07-01,315793,0.008696
3,2017-10-01,351737,0.006801
4,2018-01-01,256783,0.006605


saved: C:\Users\maria\credit-risk\reports\eda_bad_rate_by_vintage.png


In [4]:
states = q("""
SELECT state, COUNT(*) n,
       AVG(CASE WHEN default_within_24m THEN 1 ELSE 0 END)::DOUBLE AS bad_rate
FROM read_parquet($1)
GROUP BY 1 ORDER BY n DESC LIMIT 25
""", str(DATA/"abt.parquet"))

channel_purpose = q("""
SELECT channel, purpose,
       COUNT(*) n,
       AVG(CASE WHEN default_within_24m THEN 1 ELSE 0 END)::DOUBLE AS bad_rate
FROM read_parquet($1)
GROUP BY 1,2 ORDER BY n DESC
""", str(DATA/"abt.parquet"))

display(states, channel_purpose.head(20))
states.to_csv(RPTS/"eda_states.csv", index=False)
channel_purpose.to_csv(RPTS/"eda_channel_purpose.csv", index=False)

,state,n,bad_rate
0,CA,1323571,0.019299
1,TX,835993,0.023210
2,FL,761278,0.028551
3,IL,476869,0.018636
4,OH,421842,0.014510
5,MI,381278,0.017777
6,NY,363920,0.028778
7,AZ,356782,0.017151
8,PA,356012,0.016002
9,GA,348512,0.022608


,channel,purpose,n,bad_rate
0,R,P,2463184,0.019688
1,R,N,2215538,0.010798
2,C,P,1749062,0.026063
3,R,C,1649022,0.020116
4,C,N,957166,0.014284
5,C,C,607502,0.024010
6,B,P,589057,0.026884
7,B,N,505808,0.012105
8,B,C,287307,0.020870
9,9,C,3,0.000000


In [5]:
def bin_table(col, bins, labels):
    sql = f"""
    WITH b AS (
      SELECT CASE
        { " ".join([f"WHEN {col}<={edge} THEN '{lab}'" for edge, lab in zip(bins[:-1], labels[:-1])]) }
        ELSE '{labels[-1]}' END AS bin,
        default_within_24m AS y
      FROM read_parquet($1)
    )
    SELECT bin, COUNT(*) n,
           AVG(CASE WHEN y THEN 1 ELSE 0 END)::DOUBLE AS bad_rate
    FROM b GROUP BY 1 ORDER BY 1
    """
    return q(sql, str(DATA/"abt.parquet"))

fico_tbl = bin_table("fico", [620,660,700,740,780], ["<=620","621-660","661-700","701-740","741-780",">780"])
ltv_tbl  = bin_table("ltv",  [60,70,80,90,97], ["<=60","61-70","71-80","81-90","91-97",">97"])
dti_tbl  = bin_table("dti",  [30,35,40,45,50], ["<=30","31-35","36-40","41-45","46-50",">50"])

display(fico_tbl, ltv_tbl, dti_tbl)

for name, tbl in [("fico", fico_tbl), ("ltv", ltv_tbl), ("dti", dti_tbl)]:
    plt.figure()
    plt.bar(tbl["bin"], tbl["bad_rate"])
    plt.title(f"Bad rate by {name} bin"); plt.xticks(rotation=45)
    savefig(f"eda_monotone_{name}.png")

fico_tbl.to_csv(RPTS/"eda_bins_fico.csv", index=False)
ltv_tbl.to_csv(RPTS/"eda_bins_ltv.csv", index=False)
dti_tbl.to_csv(RPTS/"eda_bins_dti.csv", index=False)

,bin,n,bad_rate
0,621-660,409008,0.067886
1,661-700,1256118,0.041864
2,701-740,2328265,0.025488
3,<=620,18654,0.075480
4,>780,7011607,0.009454


,bin,n,bad_rate
0,61-70,1676412,0.015766
1,71-80,3689141,0.018524
2,81-90,1163409,0.024384
3,<=60,2877182,0.010217
4,>97,1617508,0.033912


,bin,n,bad_rate
0,31-35,1750075,0.016007
1,36-40,1929302,0.022093
2,41-45,2220084,0.032034
3,<=30,3758143,0.008603
4,>50,1366048,0.024377


saved: C:\Users\maria\credit-risk\reports\eda_monotone_fico.png
saved: C:\Users\maria\credit-risk\reports\eda_monotone_ltv.png
saved: C:\Users\maria\credit-risk\reports\eda_monotone_dti.png


In [6]:
for col in ["fico","ltv","cltv","dti","orig_rate","orig_upb"]:
    d = q(f"""
        SELECT {col}
        FROM read_parquet($1)
        WHERE {col} IS NOT NULL
    """, str(DATA/"abt_train.parquet"))
    plt.figure()
    plt.hist(d[col], bins=60)
    plt.title(f"{col} (train)")
    savefig(f"eda_hist_train_{col}.png")

saved: C:\Users\maria\credit-risk\reports\eda_hist_train_fico.png
saved: C:\Users\maria\credit-risk\reports\eda_hist_train_ltv.png
saved: C:\Users\maria\credit-risk\reports\eda_hist_train_cltv.png
saved: C:\Users\maria\credit-risk\reports\eda_hist_train_dti.png
saved: C:\Users\maria\credit-risk\reports\eda_hist_train_orig_rate.png
saved: C:\Users\maria\credit-risk\reports\eda_hist_train_orig_upb.png


In [7]:
def psi_from_parquet(feature: str, base_file: Path, cmp_file: Path, bins: int = 10):
    # get deciles on base
    base = q(f"SELECT {feature} FROM read_parquet($1) WHERE {feature} IS NOT NULL", str(base_file))[feature]
    cmp  = q(f"SELECT {feature} FROM read_parquet($1) WHERE {feature} IS NOT NULL", str(cmp_file))[feature]
    qs = np.quantile(base, np.linspace(0,1,bins+1))
    # bin counts
    base_bin = np.histogram(base, bins=qs)[0].astype(float); base_bin /= base_bin.sum()
    cmp_bin  = np.histogram(cmp,  bins=qs)[0].astype(float); cmp_bin  /= cmp_bin.sum()
    # avoid zeros
    base_bin = np.clip(base_bin, 1e-6, None); cmp_bin = np.clip(cmp_bin, 1e-6, None)
    return float(np.sum((base_bin - cmp_bin) * np.log(base_bin / cmp_bin)))

features = ["fico","ltv","cltv","dti","orig_rate","orig_upb"]
psi_rows = [{"feature": f, "psi": psi_from_parquet(f, DATA/"abt_train.parquet", DATA/"abt_test.parquet")} for f in features]
psi_df = pd.DataFrame(psi_rows).sort_values("psi", ascending=False)
display(psi_df)
psi_df.to_csv(RPTS/"eda_psi.csv", index=False)

,feature,psi
4,orig_rate,3.092030
5,orig_upb,0.261140
2,cltv,0.141451
1,ltv,0.136192
0,fico,0.016508
3,dti,0.016449


In [8]:
import json
audit = json.loads((RPTS/"audit_abt.json").read_text())
miss_top = pd.DataFrame(audit["missing_top"])
outliers  = pd.DataFrame(audit["outlier_flags"])
display(miss_top, outliers)
miss_top.to_csv(RPTS/"eda_missing_top.csv", index=False)
outliers.to_csv(RPTS/"eda_outlier_flags.csv", index=False)

,col,nulls,miss_rate
0,orig_date,11023652,1.000000
1,first_90dpd,10650316,0.966133
2,msa,1023883,0.092881
3,first_payment_date,0,0.000000
4,fico,0,0.000000
5,orig_rate,0,0.000000
6,ltv,0,0.000000
7,cltv,0,0.000000
8,orig_upb,0,0.000000
9,loan_id,0,0.000000


,feature,count_flagged
0,dti,54939
1,fico,1857
2,ltv,345
3,cltv,200
4,loan_term,29
5,orig_rate,0
6,orig_upb,0
7,num_units,0


In [ ]:
from pathlib import Path
import zipfile

here = Path.cwd().resolve()
ROOT = next((r for r in (here, *here.parents) if (r / "data" / "processed").exists()), None)
if ROOT is None:
    raise FileNotFoundError(f"Couldn't find project root from {here}")

RPTS = ROOT / "reports"
RPTS.mkdir(parents=True, exist_ok=True)

candidates = list((ROOT / "notebooks").glob("**/01_eda_duckdb.ipynb"))
if not candidates:
    raise FileNotFoundError("Could not find 01_eda_duckdb.ipynb under notebooks/")
NB = candidates[0]  # first match

keep = [
    "eda_vintage.csv",
    "eda_bad_rate_by_vintage.png",
    "eda_bins_fico.csv", "eda_bins_ltv.csv", "eda_bins_dti.csv",
    "eda_monotone_fico.png", "eda_monotone_ltv.png", "eda_monotone_dti.png",
    "eda_psi.csv",
    "eda_missing_top.csv", "eda_outlier_flags.csv",
    "eda_states.csv", "eda_channel_purpose.csv",
    "eda_hist_train_fico.png", "eda_hist_train_ltv.png",
]

out_zip = RPTS / "eda_bundle.zip"
with zipfile.ZipFile(out_zip, "w", compression=zipfile.ZIP_DEFLATED) as z:
    z.write(NB, arcname="01_eda_duckdb.ipynb")
    for f in keep:
        p = RPTS / f
        if p.exists():
            z.write(p, arcname=f)

print("Wrote:", out_zip.resolve())
print("Included notebook:", NB)

Wrote: C:\Users\maria\credit-risk\reports\eda_bundle.zip
Included notebook: C:\Users\maria\credit-risk\notebooks\notebooks\01_eda_duckdb.ipynb
